# 텍스트 전처리 (Text Preprocessing)

*   텍스트를 자연어 처리를 위해 용도에 맞도록 사전에 표준화 하는 작업
*   텍스트 내 정보를 유지하고, 중복을 제거하여 분석 효율성을 높이기 위해 전처리를 수행



### 1) 토큰화 (Tokenizing)
* 텍스트를 자연어 처리를 위해 분리 하는 것을
* 토큰화는 단어별로 분리하는 "단어 토큰화(Word Tokenization)"와 문장별로 분리하는 "문장 토큰화(Sentence Tokenization)"로 구분

(이후 실습에서는 단어 토큰화를 "토큰화"로 통일하여 칭하도록 한다)

### 2) 품사 부착(PoS Tagging)
* 각 토큰에 품사 정보를 추가
* 분석시에 불필요한 품사를 제거하거나 (예. 조사, 접속사 등) 필요한 품사를 필터링 하기 위해 사용

### 3) 개체명 인식 (NER, Named Entity Recognition)
* 각 토큰의 개체 구분(기관, 인물, 지역, 날짜 등) 태그를 부착
* 텍스트가 무엇과 관련되어있는지 구분하기 위해 사용
* 예를 들어, 과일의 apple과 기업의 apple을 구분하는 방법이 개체명 인식임

### 4) 원형 복원 (Stemming & Lemmatization)
* 각 토큰의 원형 복원을 함으로써 토큰을 표준화하여 불필요한 데이터 중복을 방지 (=단어의 수를 줄일수 있어 연산을 효율성을 높임)
* 어간 추출(Stemming) : 품사를 무시하고 규칙에 기반하여 어간을 추출
* 표제어 추출 (Lemmatization) : 품사정보를 유지하여 표제어 추출

In [2]:
import MeCab
mecab = MeCab.Tagger()
print(mecab.parse(u'영등포구청역에 있는 맛집 좀 알려주세요.'))


영등포구청역	NNP,지명,T,영등포구청역,Compound,*,*,영등포/NNP/지명+구청/NNG/장소+역/NNG/*
에	JKB,*,F,에,*,*,*,*
있	VV,*,T,있,*,*,*,*
는	ETM,*,T,는,*,*,*,*
맛집	NNG,*,T,맛집,Compound,*,*,맛/NNG/*+집/NNG/*
좀	MAG,성분부사|정도부사,T,좀,*,*,*,*
알려	VV+EC,*,F,알려,Inflect,VV,EC,알리/VV/*+어/EC/*
주	VX,*,F,주,*,*,*,*
세요	EP+EF,*,F,세요,Inflect,EP,EF,시/EP/*+어요/EF/*
.	SF,*,*,*,*,*,*,*
EOS



In [8]:
from konlpy.tag import Komoran, Kkma, Mecab
komoran = Kkma()
mecab = Mecab()


print(mecab.pos(u'영등포구청역에 있는 맛집 좀 알려주세요.'))
# print(komoran.nouns(u'영등포구청역에 있는 맛집 좀 알려주세요.'))


[('영등포구청역', 'NNP'), ('에', 'JKB'), ('있', 'VV'), ('는', 'ETM'), ('맛집', 'NNG'), ('좀', 'MAG'), ('알려', 'VV+EC'), ('주', 'VX'), ('세요', 'EP+EF'), ('.', 'SF')]


### 5) 불용어 처리 (Stopword)
* 자연어 처리를 위해 불필요한 요소를 제거하는 작업
* 불필요한 품사를 제거하는 작업과 불필요한 단어를 제거하는 작업으로 구성
* 불필요한 토큰을 제거함으로써 연산의 효율성을 높임



---



# 1 영문 전처리 실습


NLTK lib (https://www.nltk.org/) 사용

## 1.1 실습용 영문기사 수집
온라인 기사를 바로 수집하여 실습데이터로 사용

https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/

In [13]:
import requests
from bs4 import BeautifulSoup

In [96]:
url = "https://www.forbes.com/sites/adrianbridgwater/2019/04/15/what-drove-the-ai-renaissance/"
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [97]:
article  = soup.select('p')
text = article[3].get_text()
text

"And yes, she does mean everybody's job from yours to mine and onward to the role of grain farmers in Egypt, pastry chefs in Paris and dog walkers in Oregon i.e. every job. We will now be able to help direct all workers’ actions and behavior with a new degree of intelligence that comes from predictive analytics, all stemming from the AI engines we will now increasingly depend upon."

## 1.2 영문 토큰화
https://www.nltk.org/api/nltk.tokenize.html

In [19]:
! pip install nltk

In [98]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\minch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [99]:
word_tokens = word_tokenize("Good muffins cost $3.88\nin New York")
print(word_tokens)

['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York']


In [100]:
from nltk.tokenize import WordPunctTokenizer
word_tokens = WordPunctTokenizer().tokenize("Good muffins cost $3.88\nin New York")
print(word_tokens)

['Good', 'muffins', 'cost', '$', '3', '.', '88', 'in', 'New', 'York']


## 1.3 영문 품사 부착 (PoS Tagging)
분리한 토큰마다 품사를 부착한다

https://www.nltk.org/api/nltk.tag.html

태크목록 : https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

In [25]:
from nltk import pos_tag
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\minch\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [101]:
# 위에서 문자열 토큰화 한 값 전달해 품사 부착한다.
tagged = pos_tag(word_tokens)
print(tagged)

[('Good', 'JJ'), ('muffins', 'NNS'), ('cost', 'VBP'), ('$', '$'), ('3', 'CD'), ('.', '.'), ('88', 'CD'), ('in', 'IN'), ('New', 'NNP'), ('York', 'NNP')]


## 1.4 개체명 인식 (NER, Named Entity Recognition)

http://www.nltk.org/api/nltk.chunk.html

In [102]:
nltk.download("words")
nltk.download("maxent_ne_chunker")

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\minch\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\minch\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [103]:
# 위에서 품사 부착한 것 가져와 개체명 인식 시킨다.
from nltk import ne_chunk
ne_token = ne_chunk(tagged)
print(ne_token)

(S
  (GPE Good/JJ)
  muffins/NNS
  cost/VBP
  $/$
  3/CD
  ./.
  88/CD
  in/IN
  (GPE New/NNP York/NNP))


## 1.5 원형 복원
각 토큰의 원형을 복원하여 표준화 한다. 
- 한국어의 경우 어렵다.

### 1.5.1 어간추출 (Stemming)

* 규칙에 기반 하여 토큰을 표준화
* ning제거, ful 제거 등

https://www.nltk.org/api/nltk.stem.html

규칙상세 : https://tartarus.org/martin/PorterStemmer/def.txt

In [104]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [105]:
ps.stem("running") # 안쓰는 부분 잘라줌

'run'

In [106]:
ps.stem("beautiful") # 안쓰는 부분 잘라줌

'beauti'

In [107]:
ps.stem('believes')

'believ'

### 1.5.2 표제어 추출 (Lemmatization)

* 품사정보를 보존하여 토큰을 표준화

http://www.nltk.org/api/nltk.stem.html?highlight=lemmatizer

In [108]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\minch\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [109]:
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()

In [110]:
wl.lemmatize("running")

'running'

In [111]:
wl.lemmatize("beautiful")

'beautiful'

In [112]:
wl.lemmatize("believes")

'belief'

## 1.6 불용어 처리 (Stopword)

In [113]:
stop_pos = ["IN", "CC", "NNP"]
from collections import Counter
tagged = pos_tag(word_tokenize(text))
Counter(tagged).most_common()

[((',', ','), 3),
 (('from', 'IN'), 3),
 (('to', 'TO'), 3),
 (('and', 'CC'), 3),
 (('in', 'IN'), 3),
 (('.', '.'), 3),
 (('job', 'NN'), 2),
 (('the', 'DT'), 2),
 (('of', 'IN'), 2),
 (('will', 'MD'), 2),
 (('now', 'RB'), 2),
 (('all', 'DT'), 2),
 (('And', 'CC'), 1),
 (('yes', 'UH'), 1),
 (('she', 'PRP'), 1),
 (('does', 'VBZ'), 1),
 (('mean', 'VB'), 1),
 (('everybody', 'NN'), 1),
 (("'s", 'POS'), 1),
 (('yours', 'NNS'), 1),
 (('mine', 'VB'), 1),
 (('onward', 'VB'), 1),
 (('role', 'NN'), 1),
 (('grain', 'NN'), 1),
 (('farmers', 'NNS'), 1),
 (('Egypt', 'NNP'), 1),
 (('pastry', 'NN'), 1),
 (('chefs', 'NNS'), 1),
 (('Paris', 'NNP'), 1),
 (('dog', 'NN'), 1),
 (('walkers', 'NNS'), 1),
 (('Oregon', 'NNP'), 1),
 (('i.e', 'NN'), 1),
 (('every', 'DT'), 1),
 (('We', 'PRP'), 1),
 (('be', 'VB'), 1),
 (('able', 'JJ'), 1),
 (('help', 'VB'), 1),
 (('direct', 'VB'), 1),
 (('workers', 'NNS'), 1),
 (('’', 'VBP'), 1),
 (('actions', 'NNS'), 1),
 (('behavior', 'NN'), 1),
 (('with', 'IN'), 1),
 (('a', 'DT'), 1

In [116]:
words = []
for tag in tagged:
    if tag[1] in stop_pos:
        words.append(tag)
words

[('And', 'CC'),
 ('from', 'IN'),
 ('and', 'CC'),
 ('of', 'IN'),
 ('in', 'IN'),
 ('Egypt', 'NNP'),
 ('in', 'IN'),
 ('Paris', 'NNP'),
 ('and', 'CC'),
 ('in', 'IN'),
 ('Oregon', 'NNP'),
 ('and', 'CC'),
 ('with', 'IN'),
 ('of', 'IN'),
 ('from', 'IN'),
 ('from', 'IN'),
 ('AI', 'NNP')]

## 1.7 영문 텍스트 전처리 종합



---



# 2 한글 전처리 실습
영문은 공백으로 토큰화가 가능하지만, 한글의 경우 품사를 고려하여 토큰화 해야한다.

## 2.1 실습용 한글기사 수집
온라인 기사를 바로 수집하여 실습데이터로 사용

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=110&oid=023&aid=0003386456


In [122]:
url = "https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=110&oid=023&aid=0003386456"
headers = {'user-agent':'Mozilla/5.0'}
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')

In [123]:

article = soup.select("#articleBodyContents")[0].text
print(article)






// flash 오류를 우회하기 위한 함수 추가
function _flash_removeCallback() {}

美 IT 기업 창업자 가운데 컴퓨터·공학 전공은 37%뿐… 예술·인문학 등 전공 다양工學·예술의 창의적 협력으로 '혁신의 규칙'을 바꾸는 것이 4차 산업혁명의 시대정신비벡 와드와 카네기멜런대 교수·하버드대 로스쿨 특별연구원교육을 삶의 최우선 순위로 두고 있는 한국의 부모들은 대학 전공 가운데 의학과 공학·과학을 중시한다. 자녀의 직업적 성공을 위해 대학 전공으로 의학과 이공계를 우선적으로 고려하는 일은 한국이 산업화 중이던 상황에선 올바른 선택이었다. 하지만 지금은 모든 것이 달라졌다. 요즘 실리콘밸리에서 확인되는 것은 4차 산업혁명 시대에는 예술과 인문학이 의학·공학만큼 중요하다는 사실이다.스티브 잡스는 자신이 대학 시절 수강했던 서체(書體) 수업이 매킨토시(애플이 1984년 발표한 개인용 컴퓨터) 개발 성공에 큰 영향을 미쳤다고 말했다. 그는 2011년 아이패드 2를 공개하면서 "애플의 DNA는 기술만으로는 충분하지 않다. 교양과 인문학이 결합한 기술이야말로 가슴 벅찬 결과를 낳을 것"이라며 예술과 디자인의 중요성을 강조했다. 이런 관점을 바탕으로 잡스는 세계 최고 가치를 인정받는 기업을 만들었고, 기술 산업의 새로운 표준까지 정했다.실리콘밸리에서 최근 뜨고 있는 스타 기업인 중에는 인문학 전공자들이 제법 많다. 구인·구직 소셜 네트워킹 서비스 기업인 링크드인(LinkedIn) 창업자 리드 호프만은 철학 석사 학위 소지자이며, 수잔 보이치키 유튜브 CEO는 역사와 문학을 전공했다. 메신저 개발 업체 슬랙(Slack)의 창업자 스튜어트 버터필드는 철학, 세계 최대 숙박 공유 기업인 에어비앤비의 설립자 브라이언 체스키는 미술을 전공했다. 중국 알리바바그룹의 마윈 회장의 학부 전공은 영어였다.내가 속해 있는 하버드대·듀크대 연구팀은 미국 IT 기업 창업자들의 92%가 학사 학위를, 47%는 석사 학위 이상을 갖고 있음을 밝혀냈다. 창업자들의

## 2.2 한글 토큰화 및 형태소 분석

한글 자연어처리기 비교

https://konlpy.org/ko/latest/morph/

In [138]:
text = "기존 가속기 설계 및 운전에 있어서 엄밀하게 취급 되지 않았던, 공간전하 효과(선형 및 비선형), x-y coupling 및  x-z coupling, 급격한 에너지 변화 등을 고려한 빔 물리 이론 모형을 개발하고, 국제 핵융합 재료조사시설(IFMIF) 개발에 적용한다"

In [139]:

from konlpy.tag import Komoran
komoran = Komoran()
tagged = komoran.morphs(text)
print(tagged)




['기존', '가속기', '설계', '및', '운전', '에', '있', '어서', '엄밀', '하', '게', '취급', '되', '지', '않', '았', '던', ',', '공간', '전하', '효과', '(', '선형', '및', '비선형', ')', ',', 'x', '-', 'y', 'coupling', '및', 'x', '-', 'z', 'coupling', ',', '급격', '하', 'ㄴ', '에너지', '변화', '등', '을', '고려', '하', 'ㄴ', '빔', '물리', '이론', '모형', '을', '개발', '하', '고', ',', '국제', '핵융합', '재료', '조사', '시설', '(', 'IFMIF', ')', '개발', '에', '적용', '하', 'ㄴ다']


In [ ]:
from konlpy.tag import Hannanum
hannanum = Hannanum()

tagged = hannanum.morphs(text)

print(tagged)

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()

tagged = kkma.morphs(text)

print(tagged)

In [ ]:
from konlpy.tag import Okt
okt = Okt()

tagged = okt.morphs(text)

print(tagged)

In [140]:

from konlpy.tag import Mecab
mecab = Mecab()

tagged = mecab.morphs(text)
print(tagged)




['기존', '가속기', '설계', '및', '운전', '에', '있', '어서', '엄밀', '하', '게', '취급', '되', '지', '않', '았', '던', ',', '공간', '전하', '효과', '(', '선형', '및', '비', '선형', ')', ',', 'x', '-', 'y', 'coupling', '및', 'x', '-', 'z', 'coupling', ',', '급격', '한', '에너지', '변화', '등', '을', '고려', '한', '빔', '물리', '이론', '모형', '을', '개발', '하', '고', ',', '국제', '핵융합', '재료', '조사', '시설', '(', 'IFMIF', ')', '개발', '에', '적용', '한다']


In [146]:
! pip install soynlp

In [147]:
from soynlp.tokenizer import LTokenizer
tokenizer = LTokenizer()
tokenizer.tokenize(text)

['기존',
 '가속기',
 '설계',
 '및',
 '운전에',
 '있어서',
 '엄밀하게',
 '취급',
 '되지',
 '않았던,',
 '공간전하',
 '효과(선형',
 '및',
 '비선형),',
 'x-y',
 'coupling',
 '및',
 'x-z',
 'coupling,',
 '급격한',
 '에너지',
 '변화',
 '등을',
 '고려한',
 '빔',
 '물리',
 '이론',
 '모형을',
 '개발하고,',
 '국제',
 '핵융합',
 '재료조사시설(IFMIF)',
 '개발에',
 '적용한다']

## 2.3 한글 품사 부착 (PoS Tagging)

PoS Tag 목록

https://docs.google.com/spreadsheets/u/1/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0

In [148]:
from konlpy.tag import Mecab
mecab = Mecab()

tagged = mecab.pos(text)

print(tagged)

[('기존', 'NNG'), ('가속기', 'NNG'), ('설계', 'NNG'), ('및', 'MAJ'), ('운전', 'NNG'), ('에', 'JKB'), ('있', 'VV'), ('어서', 'EC'), ('엄밀', 'XR'), ('하', 'XSA'), ('게', 'EC'), ('취급', 'NNG'), ('되', 'VV'), ('지', 'EC'), ('않', 'VX'), ('았', 'EP'), ('던', 'ETM'), (',', 'SC'), ('공간', 'NNG'), ('전하', 'NNG'), ('효과', 'NNG'), ('(', 'SSO'), ('선형', 'NNG'), ('및', 'MAJ'), ('비', 'XPN'), ('선형', 'NNG'), (')', 'SSC'), (',', 'SC'), ('x', 'SL'), ('-', 'SY'), ('y', 'SL'), ('coupling', 'SL'), ('및', 'MAJ'), ('x', 'SL'), ('-', 'SY'), ('z', 'SL'), ('coupling', 'SL'), (',', 'SC'), ('급격', 'XR'), ('한', 'XSA+ETM'), ('에너지', 'NNG'), ('변화', 'NNG'), ('등', 'NNB'), ('을', 'JKO'), ('고려', 'NNG'), ('한', 'XSV+ETM'), ('빔', 'NNG'), ('물리', 'NNG'), ('이론', 'NNG'), ('모형', 'NNG'), ('을', 'JKO'), ('개발', 'NNG'), ('하', 'XSV'), ('고', 'EC'), (',', 'SC'), ('국제', 'NNG'), ('핵융합', 'NNG'), ('재료', 'NNG'), ('조사', 'NNG'), ('시설', 'NNG'), ('(', 'SSO'), ('IFMIF', 'SL'), (')', 'SSC'), ('개발', 'NNG'), ('에', 'JKB'), ('적용', 'NNG'), ('한다', 'XSV+EC')]


## 2.4 불용어(Stopword) 처리
분석에 불필요한 품사를 제거하고, 불필요한 단어(불용어)를 제거한다

In [149]:
stopwords = []
with open("stopwords.txt") as f:
    for line in f:
        stopwords.append(line.strip())
len(stopwords)

675

In [150]:
stop_pos = ['EC', 'EP', 'SC', 'JK', 'JKO']

In [151]:
print(len(tagged))
for tag in tagged:
    if tag[0] in stopwords or tag[1] in stop_pos:
        continue
    words.append(tag[0])
print(len(words))

print(len(tagged))


67
56


# 2 N-gram